<a href="https://colab.research.google.com/github/EmperorNao/MLHub/blob/main/DLS/sem1/HW3/AlmostPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!wget -O train.csv https://vk.com/doc209646771_632063537?hash=72705ee81cb683817b&dl=474cf35459f17f62ca
!wget -O test.csv https://vk.com/doc209646771_632063527?hash=59a58055c24a70d69f&dl=cbee47200e26df8667
!wget -O submission.csv https://vk.com/doc209646771_632063530?hash=c117ba1d00a374c43b&dl=e4ed277865d5bed7f0

--2022-03-07 06:08:46--  https://vk.com/doc209646771_632063537?hash=72705ee81cb683817b
Resolving vk.com (vk.com)... 87.240.137.158, 87.240.190.67, 87.240.190.72, ...
Connecting to vk.com (vk.com)|87.240.137.158|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://psv4.userapi.com/c237231/u209646771/docs/d38/1e867e791f53/train.csv?extra=GWalBNrwFyAzLJxYgzDjb724xlBs9OxXkPue2jXCbAGSFrRA6Qc4s3Gd_M558B9Ykhx_RVbU0JjvDNUzTj-IAFk04nFFA1bfMmQ_fKw0IO4zby5j6LSJnFNBL-OmEhRBsz6nycUXtdXmlg&dl=1 [following]
--2022-03-07 06:08:47--  https://psv4.userapi.com/c237231/u209646771/docs/d38/1e867e791f53/train.csv?extra=GWalBNrwFyAzLJxYgzDjb724xlBs9OxXkPue2jXCbAGSFrRA6Qc4s3Gd_M558B9Ykhx_RVbU0JjvDNUzTj-IAFk04nFFA1bfMmQ_fKw0IO4zby5j6LSJnFNBL-OmEhRBsz6nycUXtdXmlg&dl=1
Resolving psv4.userapi.com (psv4.userapi.com)... 87.240.190.76, 87.240.190.89, 87.240.137.134, ...
Connecting to psv4.userapi.com (psv4.userapi.com)|87.240.190.76|:443... connected.
HTTP request sent, awaiting res

In [10]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from copy import copy

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

from catboost.metrics import AUC
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=UserWarning)



In [42]:
SEED = 42
np.random.seed(42)

In [43]:
class FeatureGenerator:


    def __init__(self, models: dict = {}):
        self.models = models
        self.new_columns = None

    def __call__(self, data, num_cols):

        self.new_columns = set()

        data, app = self.__pairs_generator(data, num_cols)
        self.new_columns.update(app)

        data, app = self.__models_generator(data)
        self.new_columns.update(app)

        copy_new_columns = copy(self.new_columns)
        self.new_columns = None
        return data, copy_new_columns

    def __models_generator(self, data):

        outp = pd.DataFrame()
        for modelname, model in self.models.items():
            outp[modelname + "_proba"] = model.predict(data)
            outp[modelname + "_output"] = model.predict_proba(data)[:, 1]

        return pd.concat([data, outp], axis=1), set(outp.columns)


    def __pairs_generator(self, data, num_cols):

        append_cols = set()
        for i, fcol in enumerate(num_cols):
            for j, scol in enumerate(num_cols) :

                newcolname = fcol + ' * ' + scol
                data[newcolname] = data[fcol] * data[scol]
                append_cols.add(newcolname)
        
        return data, append_cols

In [44]:
def preprocess(fname, num_cols, cat_cols, feature_cols, train=True, generator = None):

    feature_cols = copy(feature_cols)
    num_cols = copy(num_cols)
    cat_cols = copy(cat_cols)

    data = pd.read_csv(fname)

    data = data.replace(' ', 0)
    data['TotalSpent'] = data['TotalSpent'].astype('float128')

    if generator:

        data, new_num_cols = generator(data, num_cols)
        num_cols.update(new_num_cols)
        feature_cols.update(new_num_cols)

    if train:
        X = data[feature_cols]
        y = data[target_col]
        return X, y, num_cols, cat_cols, feature_cols
    else:
        X = data[feature_cols]
        return X

In [45]:
def build_estimator(estimator, num_cols, cat_cols, feature_cols, cb=False):

    if cb:
        return estimator

    else:
        numeric_transformer = Pipeline(
            steps=[("scaler", StandardScaler())]
        )

        categorical_transformer = OneHotEncoder(handle_unknown="ignore")

        preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, list(num_cols)),
            ("cat", categorical_transformer, list(cat_cols))]
        )

        return Pipeline(
            steps=[("preprocessor", preprocessor), ("classifier", estimator)]
        )

In [46]:
data = pd.read_csv('./train.csv')

In [47]:
num_cols = set([
    'ClientPeriod',
    'MonthlySpending',
 'TotalSpent'
])

cat_cols = set([
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
])

feature_cols = num_cols.union(cat_cols)
target_col = 'Churn'

In [ ]:
X, y, new_num_cols, new_cat_cols, new_feature_cols = preprocess("./train.csv", num_cols, cat_cols, feature_cols, generator = FeatureGenerator())

In [ ]:
lm = build_estimator(
    LogisticRegression(random_state=SEED, solver='saga', max_iter=1e5, penalty = 'elasticnet', C=1e2, class_weight='balanced', l1_ratio = 0.1),
    new_num_cols, new_cat_cols, new_feature_cols)

In [ ]:
lm.fit(X, y);

In [ ]:
X, y, new_num_cols, new_cat_cols, new_feature_cols = preprocess("./train.csv", num_cols, cat_cols, feature_cols, generator = FeatureGenerator({'lm': lm}))

In [ ]:
catboost = build_estimator(
    CatBoostClassifier(l2_leaf_reg = 4, depth = 6, learning_rate = 0.01, auto_class_weights = 'Balanced', eval_metric=AUC(), random_seed=SEED, cat_features=list(new_cat_cols)),
    new_num_cols, new_cat_cols, new_feature_cols, cb=True)

In [ ]:
catboost.fit(X, y);

0:	total: 42.2ms	remaining: 42.1s
1:	total: 83.7ms	remaining: 41.8s
2:	total: 145ms	remaining: 48s
3:	total: 185ms	remaining: 46.1s
4:	total: 226ms	remaining: 44.9s
5:	total: 264ms	remaining: 43.7s
6:	total: 318ms	remaining: 45.1s
7:	total: 370ms	remaining: 45.9s
8:	total: 422ms	remaining: 46.4s
9:	total: 461ms	remaining: 45.6s
10:	total: 511ms	remaining: 45.9s
11:	total: 549ms	remaining: 45.2s
12:	total: 592ms	remaining: 44.9s
13:	total: 640ms	remaining: 45.1s
14:	total: 679ms	remaining: 44.6s
15:	total: 692ms	remaining: 42.6s
16:	total: 736ms	remaining: 42.6s
17:	total: 778ms	remaining: 42.4s
18:	total: 834ms	remaining: 43.1s
19:	total: 874ms	remaining: 42.8s
20:	total: 929ms	remaining: 43.3s
21:	total: 978ms	remaining: 43.5s
22:	total: 1.02s	remaining: 43.6s
23:	total: 1.1s	remaining: 44.6s
24:	total: 1.11s	remaining: 43.2s
25:	total: 1.14s	remaining: 42.5s
26:	total: 1.17s	remaining: 42.2s
27:	total: 1.19s	remaining: 41.3s
28:	total: 1.21s	remaining: 40.6s
29:	total: 1.29s	remainin

KeyboardInterrupt: ignored

In [ ]:
X_test = preprocess("./test.csv", num_cols, cat_cols, feature_cols, train=False, generator=FeatureGenerator({"lm": lm}))

submission = pd.read_csv('./submission.csv', )
submission['Churn'] = catboost.predict_proba(X_test)[:, 1]
submission.to_csv('./my_submission.csv', index=False)

In [220]:
parameters = {
    'classifier__penalty': ['elasticnet'],
    'classifier__C': [1e2],
    'classifier__class_weight': [None],
    'classifier__l1_ratio': [0.1],
}

In [221]:
pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(random_state=SEED, solver='saga'))]
)

In [222]:
search = GridSearchCV(pipeline, parameters, scoring = 'roc_auc', verbose=1, n_jobs =-1)
search.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['ClientPeriod',
                                                                          'MonthlySpending',
                                                                          'TotalSpent']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Sex',
                                                                          'IsSeniorCitizen',
             

In [223]:
clf = search.best_estimator_
print(f"Лучший скор: {search.best_score_}")
print(f"Лучшие параметры: {search.best_params_}")

Лучший скор: 0.844827370696579
Лучшие параметры: {'classifier__C': 100.0, 'classifier__class_weight': None, 'classifier__l1_ratio': 0.1, 'classifier__penalty': 'elasticnet'}


roc = *0.8448*

In [224]:
print(f"Лучший скор: {roc_auc_score(y_test, clf.predict(X_test))}")

Лучший скор: 0.7213257843831092


CB

In [247]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [249]:
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold

In [256]:
superCB = CatBoostClassifier(eval_metric=AUC(), random_seed=SEED, cat_features=list(cat_cols))

In [257]:
parameters = {
    'depth': [6],
    'learning_rate': [0.01],
    'auto_class_weights': ['Balanced']
}

In [258]:
from google.colab import output
output.enable_custom_widget_manager()

In [259]:
grid_search_result = superCB.grid_search(parameters, cv=5, X=X, y=y, plot=True, partition_random_seed=SEED)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Выходные данные были обрезаны до нескольких последних строк (5000).
24:	test: 0.8267657	best: 0.8271383 (15)	total: 475ms	remaining: 18.5s
25:	test: 0.8266940	best: 0.8271383 (15)	total: 491ms	remaining: 18.4s
26:	test: 0.8263283	best: 0.8271383 (15)	total: 510ms	remaining: 18.4s
27:	test: 0.8266338	best: 0.8271383 (15)	total: 534ms	remaining: 18.5s
28:	test: 0.8263792	best: 0.8271383 (15)	total: 554ms	remaining: 18.6s
29:	test: 0.8263515	best: 0.8271383 (15)	total: 573ms	remaining: 18.5s
30:	test: 0.8266153	best: 0.8271383 (15)	total: 591ms	remaining: 18.5s
31:	test: 0.8269439	best: 0.8271383 (15)	total: 599ms	remaining: 18.1s
32:	test: 0.8266338	best: 0.8271383 (15)	total: 611ms	remaining: 17.9s
33:	test: 0.8266893	best: 0.8271383 (15)	total: 629ms	remaining: 17.9s
34:	test: 0.8264464	best: 0.8271383 (15)	total: 646ms	remaining: 17.8s
35:	test: 0.8269231	best: 0.8271383 (15)	total: 665ms	remaining: 17.8s
36:	test: 0.8270110	best: 0.8271383 (15)	total: 682ms	remaining: 17.7s
37:	test:

In [321]:
print(f"Лучший скор: {roc_auc_score(y_test, superCB.predict(X_test))}")

Лучший скор: 0.7913658881811748


In [260]:
best_model = superCB

In [264]:
X_test = preprocess("./test.csv", train=False)

submission = pd.read_csv('./submission.csv')
submission['Churn'] = best_model.predict_proba(X_test)[:, 1]
submission.to_csv('./my_submission.csv', index=False)